 #### <u>Lab 10</u>
##### CSCI316 - Big Data

**Group Members**
- Hunter Riddle (6451159)
- Oliver Small (6443175)

#### Plain Tensorflow MLP Implementation

In [10]:
# imports
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings

# ignore TF warnings
tf.logging.set_verbosity(tf.logging.ERROR)

AttributeError: module 'tensorflow' has no attribute 'logging'

In [ ]:
# read in data
data = pd.read_csv("abalone.data", sep=',', header=None)
n_rows = data.shape[0]
n_columns = data.shape[1]

In [ ]:
# format data
data.columns = ['sex', 'length', 'diameter', 'height', 'whole_weight', 'shucked_weight', 'viscera_weight', 'shell_weight', 'rings']

# one hot encoding ('sex' attribute)
for (num, value) in enumerate(set(data['sex']), start=0):
    data['sex'] = data['sex'].replace([value], num)

In [ ]:
# create testsets (holdout evaluation method, 70/30 split)
X_train = data.sample(int(n_rows*0.7))
X_test = data.drop(X_train.index)

# reset indexes
X_train = X_train.reset_index()
X_test = X_test.reset_index()

# cleaning up
y_train = np.array(X_train['rings'])
y_test = np.array(X_test['rings'])
X_train = np.array(X_train.iloc[: ,1:-1])
X_test = np.array(X_test.iloc[: ,1:-1])

In [ ]:
# MLP hyperparameters
n_features = 8
n_neurons_hidden = 10
n_neurons_out = 1

In [ ]:
# MLP variables
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
y_true = tf.placeholder(tf.float32, shape=(None), name="y_true")

W1 = tf.get_variable("weights1", dtype=tf.float32, initializer=tf.zeros((n_features, n_neurons_hidden)))
b1 = tf.get_variable("bias1", dtype=tf.float32, initializer=tf.zeros((n_neurons_hidden)))
W2 = tf.get_variable("weights2", dtype=tf.float32, initializer=tf.zeros((n_neurons_hidden, n_neurons_out)))
b2 = tf.get_variable("bias2", dtype=tf.float32, initializer=tf.zeros((n_neurons_out)))

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [ ]:
# create network
h = tf.nn.sigmoid(tf.matmul(X, W1) + b1)
z = tf.matmul(h, W2) + b2

In [ ]:
# define costs
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=z, labels=y_true)
cost = tf.reduce_mean(cross_entropy)

# define accuracy
#accuracy = tf.metrics.accuracy(labels=tf.squeeze(set(y_test)), predictions=tf.argmax(z, axis=1),)[1]

In [ ]:
# training
learning_rate = 0.1
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_op = optimizer.minimize(cost)

In [ ]:
# execution
init = tf.global_variables_initializer()
n_epochs = 40
with tf.Session() as session:
    init.run()
    for epoch in range(n_epochs):
        session.run(training_op, feed_dict={
            X: X_train,
            y_true: y_train
        })
        
    predictions = session.run(z, feed_dict={'X':X_test})
    print(predictions[33:57])
   

#### Keras API in Tensorflow MLP Implementation

In [ ]:
# imports

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# read in data
data = pd.read_csv("abalone.data", sep=',', header=None)
n_rows = data.shape[0]
n_columns = data.shape[1]


In [ ]:
# format data
data.columns = ['sex', 'length', 'diameter', 'height', 'whole_weight', 'shucked_weight', 'viscera_weight', 'shell_weight', 'rings']

# one hot encoding ('sex' attribute)
for (num, value) in enumerate(set(data['sex']), start=0):
    data['sex'] = data['sex'].replace([value], num)

In [ ]:
# create testsets (holdout evaluation method, 70/30 split)
train_features = data.sample(int(n_rows*0.7))
test_features = data.drop(train_features.index)

train_labels = train_features.pop('rings')
test_labels = test_features.pop('rings')


In [ ]:
# create normalizer
normalizer = tf.keras.layers.Normalization(axis=-1)


In [ ]:
# Create DNN model
model = keras.Sequential([
    normalizer,
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.001))



In [ ]:
# train the model
%%time
history = model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)


Wall time: 4.6 s


In [ ]:
# get model summary
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_1 (Normalizati (None, 8)                 17        
_________________________________________________________________
dense_27 (Dense)             (None, 64)                576       
_________________________________________________________________
dense_28 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 65        
Total params: 4,818
Trainable params: 4,801
Non-trainable params: 17
_________________________________________________________________


In [ ]:
# make predictions

test_predictions = model.predict(test_features).flatten()

error = test_predictions - test_labels